# DynaCat 动态催化 101

随着实验技术和科学计算的发展，越来越多的原位谱学、显微学表征以及电子结构理论计算研究表明，催化剂结构在反应原位条件下会经历动态的演化。传统针对单一、静态结构模型的第一性原理计算将难以满足日益复杂的催化科学问题。为开展动态催化研究，我们研发了动态催化智能计算工作流软件实现了对反应势函数的自动化构建以及催化性质的精确计算，并将主要工具以 Bohrium App 的形式发布以方便大家使用。

此 Notebook 将展示如何使用 Bohrium App 和 Notebook 实现完整的动态催化计算及分析。

动态催化研究的计算流程如图所示：

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/9f708e89fc9d41329e45e2b3f0d7c33d/3c14b558-74a7-4ddc-9f93-bd2bb3cc4dad.png" width="50%">

其主要流程包括
* 使用 AIMD 从初始结构出发构造势函数训练所需的启动数据。
* 使用增强采样和主动学习工作流进行势函数训练。
* 使用训练完备的势函数计算催化体系的自由能。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/d1c8934e3c5243d5936d17cf23882e2f/a29a2ae8-f55d-49a0-b7a1-8e9307061e30.png" width="50%">

完成上流程所需要用到的 Bohrium App 包括
* [CP2K Lightning](https://bohrium.dp.tech/apps/cp2k-lightning): 提供结构并简单进行配置，即可快速生成和运行基于 CP2K 的 AIMD 计算，用于生成初始搜索结构和初始训练数据集。
* [DynaCat TESLA](https://bohrium.dp.tech/apps/dynacat-tesla): 根据初始搜索结构和训练数据集运行主动学习工作流，自动根据预置的训练策略，实现 DeepMD 势函数的自动训练。
* [DynaCat MD](https://bohrium.dp.tech/apps/dynacat-md): 使用训练完毕的 DeepMD 势函数执行分子动力学模拟，得到催化体系性质。

相关应用代码均开源并托管于[dflow-galaxy](https://github.com/chenggroup/dflow-galaxy) 项目，
欢迎大家提交 Issues 或 PR 参与到开源社区的开发中。

本示例将根据一个预先准备好的 Ag13O2 体系，演示如何使用上述工具分析O-O解离过程的自由能。
所用的结构文件可以从[此处](https://github.com/chenggroup/dflow-galaxy/tree/main/notebook/dynacat)下载。

您可以直接在 Borhium 平台上直接运行此 Notebook, 也可以将其下载到本地环境运行。

由于接下来我们会使用 ai2-kit 提供的命令行工具对输出的数据进行处理以满足不同应用的输入要求，因此请确保 ai2-kit 正确安装。

In [ ]:
!pip install -U ai2-kit

## 使用 CP2K Lightning 生成启动数据

打开 CP2K Lighting 应用的页面，将下载的POSCAR文件上传到 `System File` 字段中，然后根据提示对参数进行配置。 
针对本示例所提供的体系，直接使用默认配置运行即可。
需要注意的是，如果要正式提交作业执行，需要取消最上方的 Dry Run 配置，否则运行完成后只会生成配置文件，不会执行具体的 AIMD 仿真。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/f76804c4b2f7402dad905bd9203e5b18/f06086bb-7817-4d9e-8bed-b9a058ee2a12.png" width="60%">

配置完毕后根据引导提交作业，接下来就只要静待作业执行完成即可。

作业执行完毕后，你会看到如图所示的输出文件

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/f76804c4b2f7402dad905bd9203e5b18/31f4f06e-b918-454f-83f0-e920e40b409d.png" width="60%">

其中我们需要关注以下文件：

* `dp-dataset`: `deepmd/npy` 格式，可直接做为 DeepMD 的训练数据。
* `aimd.xyz`: `extxyz` 格式的轨迹文件。
* `cp2k.dft.inp`: 用于 CP2K DFT 标注的配置模板，可被 DeepMD TESLA 直接使用。

由于在接下来的步骤中我们会使用到上述文件，请将其下载到本地文件中。

## 使用 DynaCat TESLA 进行势函数训练

在得到启动数据后，我们还需要对他们进行简单的处理以在 DynaCat TESLA中使用。

由 CP2K Lightning 生成的 `dp-dataset.zip` 无需处理即可直接做为 DeepMD 的初始训练数据集。 `cp2k.dft.inp` 文件可直接用作 CP2K 输入模板。

初始结构可以从上述AIMD生成的结构文件 `aimd.xyz` 中抽取 10 帧即可，该操作可以使用一行 `ai2-kit` 命令行快速实现。

您可以将 `aimd.xyz` 上传至 Notebook 中处理，如果您在本地电脑上安装了 `ai2-kit`, 也可以直接在本地运行。

In [ ]:
!ai2-kit tool ase read /personal/aimd.xyz - sample 10 - write /personal/ag13o2-init.xyz

如果您在 Bohiurm 平台上处理，请将文件输出至 `/personal` 目录，该目录中的文件可从 Borhrium App 直接访问。

接下来，我们还需要准备用于增强采样的 PLUMED 配置文件，本示例中关注的是体系中 O-O 原子的解离过程, 可使用以下配置：

```bash
UNITS LENGTH=A
# define reaction coordinates, e.g. CV1, CV2, ...
# you may define as many as you want
CV1: DISTANCE ATOMS=14,15

# define sampling method: metadynamics
metad: METAD ARG=CV1 SIGMA=0.1 HEIGHT=5 PACE=100 FILE=HILLS

# define more commands if you need

# print CVs
PRINT STRIDE=10 ARG=CV1,metad.bias FILE=COLVAR
```

其中 CV1 指定的 ATOMS 14, 15 为体系中氧原子在LAMMPS中的序号。

通常我们还需要准备一个 DeepMD 配置模板文件，不过此例使用应用缺省的提供的配置即可。
如您需要对配置进行修改，可以从[此处](https://github.com/chenggroup/dflow-galaxy/tree/main/dflow_galaxy/res/dynacat)下载默认配置进行修改。

上述准备完成后即可打开 DynaCat TESLA 界面，根据界面引导依次上传文件或者从 Bohrium Workspace 中选择。

提交作业时请注意以下事项
* 取消 Dry Run, 否则该作业只会生成配置不会真正执行
* 将上述 PLUMED 配置粘贴至对应的字段中

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/d1c8934e3c5243d5936d17cf23882e2f/1c358ffa-a107-4cf8-8a60-a5072d14e7e8.png" width="60%">

作业提交后等待其完成，完成后可以在报告中看到每轮训练的收敛情况。
您可以在输出中搜索 `frozen_model.pb`, 从最后一次迭代中下载对应的势函数模型用于后续的计算。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/d1c8934e3c5243d5936d17cf23882e2f/0a710620-9d09-4edf-8fa9-eb18543075ed.png" width="60%">

## 使用 DynaCat MD 进行催化性质探索

该步骤使用已训练完成的势函数完成最后的性质计算，其配置项与上述势函数训练过程中所使用的配置基本相同，
需要注意的配置包括
* 取消 Dry Run 选项，否则只会生成配置不会执行具体作业。
* DeepMD 势函数：使用最后一轮训练生成的 `frozen_model.pb` 文件。
* PLUMED 配置：使用与上述训练相同的配置内容。

其它选项如温度等根据研究目的进行选择。 

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/d1c8934e3c5243d5936d17cf23882e2f/00a60c9c-23f4-4361-b2b6-60e4456d0727.png" width="60%">

计算完成即可在报告中查看到如图所示的性质。
如希望使用自定义算法进行分析亦可选择将数据下载至Notebook进行进一步的计算。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13325/d1c8934e3c5243d5936d17cf23882e2f/cb65be3e-fd14-4cdb-981b-96495829113e.png" width="60%">

## 结语

以上我们通过一个简单的案例演示如何使用一系列的Bohrium App 完成动态催化的研究。
相比于传统需要在本地部署和运行工作流，使用Bohrium 提供的平台真正实现了开箱可用的计算。

以上所有内容均以开源项目的形式托管在GitHub, 欢迎有兴趣的同学一起投身到开源科学应用的开发中！
